In [1]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

In [2]:
import dask.dataframe as dd
from dask.distributed import Client

In [3]:
client = Client(n_workers=12, memory_limit='16GB')

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 178.81 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60122,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 178.81 GiB
Comm: tcp://127.0.0.1:60203,Total threads: 1
Dashboard: http://127.0.0.1:60207/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:60125,


In [5]:
df_encoded_sampled = dd.read_csv(r'df_encoded_sampled.csv', blocksize = '80MB')
df_encoded_sampled

,ts,flow_duration,Header_Length,Source Port,Destination Port,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,urg_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,urg_count,rst_count,max_duration,min_duration,sum_duration,average_duration,std_duration,CoAP,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,MAC,Magnitue,Radius,Covariance,Variance,Weight,DS status,Fragments,Sequence number,Protocol Version,flow_idle_time,flow_active_time,label,subLabel,subLabelCat,Source IP_0,Source IP_100.21.15.118,Source IP_100.21.156.247,Source IP_131.202.255.192,Source IP_15.204.8.106,Source IP_15.204.8.15,Source IP_15.204.8.160,Source IP_15.204.8.196,Source IP_15.204.8.203,Source IP_15.204.8.206,Source IP_15.204.8.22,Source IP_15.204.8.229,Source IP_15.204.8.86,Source IP_15.204.9.158,Source IP_15.204.9.23,Source IP_15.204.9.232,Source IP_15.204.9.241,Source IP_15.204.9.3,Source IP_15.204.9.4,Source IP_15.204.9.8,Source IP_159.203.8.72,Source IP_172.16.63.128,Source IP_172.16.64.128,Source IP_172.16.65.128,Source IP_172.16.66.128,Source IP_172.16.67.128,Source IP_174.119.13.119,Source IP_192.168.1.84,Source IP_192.168.219.134,Source IP_192.168.219.2,Source IP_192.168.4.2,Source IP_192.168.4.3,Source IP_198.164.163.50,Source IP_200.200.200.128,Source IP_34.219.85.253,Source IP_35.161.158.54,Source IP_35.164.195.39,Source IP_44.229.160.13,Source IP_44.230.212.240,Source IP_44.231.176.168,Source IP_44.235.194.199,Source IP_44.240.74.103,Source IP_46.235.230.122,Source IP_46.235.231.111,Source IP_51.81.202.205,Source IP_51.81.223.249,Source IP_52.12.191.236,Source IP_52.92.243.210,Source IP_54.188.109.168,Source IP_54.200.124.31,Source IP_54.218.202.245,Source IP_54.68.229.105,Source IP_54.69.37.156,Source IP_8.8.4.4,Source IP_8.8.8.8,Source IP_93.93.128.193,Source IP_93.93.135.141,Destination IP_0,Destination IP_100.21.15.118,Destination IP_100.21.156.247,Destination IP_131.202.255.192,Destination IP_15.204.8.106,Destination IP_15.204.8.15,Destination IP_15.204.8.160,Destination IP_15.204.8.196,Destination IP_15.204.8.203,Destination IP_15.204.8.206,Destination IP_15.204.8.22,Destination IP_15.204.8.229,Destination IP_15.204.8.86,Destination IP_15.204.9.158,Destination IP_15.204.9.23,Destination IP_15.204.9.232,Destination IP_15.204.9.241,Destination IP_15.204.9.3,Destination IP_15.204.9.4,Destination IP_15.204.9.8,Destination IP_159.203.8.72,Destination IP_172.16.63.128,Destination IP_172.16.63.193,Destination IP_172.16.64.128,Destination IP_172.16.65.1,Destination IP_172.16.65.254,Destination IP_172.16.66.128,Destination IP_172.16.67.128,Destination IP_174.119.13.119,Destination IP_185.125.190.17,Destination IP_185.125.190.18,Destination IP_185.125.190.48,Destination IP_185.125.190.49,Destination IP_192.112.36.4,Destination IP_192.168.219.2,Destination IP_192.168.4.2,Destination IP_192.168.4.3,Destination IP_192.203.230.10,Destination IP_192.33.4.12,Destination IP_192.36.148.17,Destination IP_192.5.5.241,Destination IP_192.58.128.30,Destination IP_198.97.190.53,Destination IP_199.7.83.42,Destination IP_199.7.91.13,Destination IP_200.200.200.254,Destination IP_202.12.27.33,Destination IP_224.0.0.22,Destination IP_224.0.0.251,Destination IP_239.255.255.250,Destination IP_34.122.121.32,Destination IP_34.219.85.253,Destination IP_35.161.158.54,Destination IP_35.164.195.39,Destination IP_35.224.170.84,Destination IP_35.232.111.17,Destination IP_44.229.160.13,Destination IP_44.230.212.240,Destination IP_44.231.176.168,Destination IP_44.235.194.199,Destination IP_44.240.74.103,Destination IP_46.235.230.122,Destination IP_46.235.231.111,Destination IP_51.81.202.205,Destination IP_51.81.223.249,Destination IP_52.12.191.236,Destination IP_52.92.243.210,Destination IP_54.188.109.168,Destination IP_54.188.99.4,Destination IP_54.200.124.31,Destination IP_54.218.202.245,Destination IP_54.68.229.105,D

In [16]:
from sklearn.decomposition import IncrementalPCA
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# Convert Dask DataFrame to Dask Array for PCA processing
df_dask_array = df_encoded_sampled.to_dask_array(lengths=True)

# Initialize IncrementalPCA
ipca = IncrementalPCA(n_components=20, batch_size=10000)  # Adjust batch size if needed




In [19]:
# Rechunk the Dask array into manageable chunk sizes for incremental PCA fitting
df_dask_array = df_dask_array.rechunk((10000, df_dask_array.shape[1]))

# Fit the PCA incrementally over each chunk
for chunk in df_dask_array.to_delayed():
    chunk_np = dd.array(chunk)  # Convert the delayed chunk to a NumPy array
    ipca.partial_fit(chunk_np)

# After fitting, transform the entire dataset
df_pca_all = ipca.transform(df_dask_array.compute())  # Compute and transform the full Dask array



AttributeError: module 'dask.dataframe' has no attribute 'array'

In [18]:
# Extract eigenvalues (explained variance)
eigenvalues = ipca.explained_variance_

# Plot the explained variance
plt.bar(range(len(eigenvalues)), eigenvalues)
plt.plot(eigenvalues, "r")
plt.plot(eigenvalues, "ro")
plt.show()

AttributeError: 'IncrementalPCA' object has no attribute 'explained_variance_'

In [ ]:
# Compute the number of rows
num_rows = df.shape[0].compute()
# Print the number of rows
print("Number of rows:", num_rows)

# Compute the number of columns
num_columns = len(df.columns)

# Print the number of columns
print("Number of columns:", num_columns)

In [ ]:
# To print the column names
print(df.columns)

In [ ]:
df.head()  # conda install -c conda-forge msgpack-python==1.0.5

In [ ]:
df.isna().sum()

In [ ]:
non_numeric_columns = df.select_dtypes(exclude=['number']).columns.tolist()

print("Columns with non-numeric values:")
print(non_numeric_columns)

In [ ]:
# Remove leading and trailing spaces from column names
df.columns = df.columns.str.strip()  

In [ ]:
Source_IP_names = df['Source IP'].unique().compute()

print("Number of Source IP Names:", len(Source_IP_names))
print("Source IP Names:")
print(Source_IP_names)

In [ ]:
Destination_IP_names = df['Destination IP'].unique().compute()

print("Number of Destination IP Names:", len(Destination_IP_names))
print("Source IP Names:")
print(Destination_IP_names)

In [ ]:
unique_protocol_names = df['Protocol_name'].unique().compute()

print("Number of Unique Protocol Names:", len(unique_protocol_names))
print("Unique Protocol Names:")
print(unique_protocol_names)

In [ ]:
label_names = df['label'].unique().compute()

print("Number of label_names:", len(label_names))
print("label_names:")
print(label_names)

In [ ]:
subLabel_names = df['subLabel'].unique().compute() 

print("Number of subLabel_names:", len(subLabel_names))
print("subLabel_names:")
print(subLabel_names)

In [ ]:
subLabelCat_names = df['subLabelCat'].unique().compute()

print("Number of subLabelCat_names:", len(subLabelCat_names))
print("subLabelCat_names:")
print(subLabelCat_names)

In [ ]:
# Step 1: Convert the relevant columns to categorical type
df = df.categorize(columns=['Source IP', 'Destination IP', 'Protocol_name'])

# Step 2: Apply one-hot encoding using Dask's get_dummies
df_encoded = dd.get_dummies(df, columns=['Source IP', 'Destination IP', 'Protocol_name'])

# Step 3: Display the first few rows of the transformed DataFrame
print(df_encoded.head())

In [ ]:
# To display the first few rows of data
df_encoded.head()  # By default, it shows the first 5 rows

In [ ]:
# Save the filtered DataFrame to a CSV file
# df_encoded.to_csv('df_encoded/*.csv', index=False, single_file=True)  # Save as a single CSV file
df_encoded.to_csv('df_encoded.csv', index=False, single_file=True)

print("Encoded Columns saved as df_encoded.csv'.")

In [ ]:
mean_values = df_encoded.mean()  # Returns a series
print(mean_values)  

In [ ]:
df_encoded.head()

num_rows = df_encoded.shape[0]
num_cols = df_encoded.shape[1]

print("Number of rows in the sample:", num_rows)
print("Number of columns in the sample:", num_cols)

In [ ]:
df_encoded.describe()